In [ ]:
from google.colab import files
import io
uploaded = files.upload()

Saving test_data15variables_dropFirst_stratified.csv to test_data15variables_dropFirst_stratified.csv
Saving train_data15variables_dropFirst_stratified.csv to train_data15variables_dropFirst_stratified.csv


In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv(io.BytesIO(uploaded['train_data15variables_dropFirst_stratified.csv']))

In [ ]:
df_test = pd.read_csv(io.BytesIO(uploaded['test_data15variables_dropFirst_stratified.csv']))

In [ ]:
X_train = df_train.drop(columns=['grav'])

In [ ]:
y_train = df_train["grav"]

In [ ]:
X_test = df_test.drop(columns=['grav'])

In [ ]:
y_test = df_test["grav"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [ ]:
y_test = le.fit_transform(y_test)

In [ ]:
import numpy as np

In [ ]:
unique_values = np.unique(y_train)
print("Valeurs uniques de y_train :", unique_values)

Valeurs uniques de y_train : [0 1 2 3]


In [ ]:
unique_values = np.unique(y_test)
print("Valeurs uniques de y_test :", unique_values)

Valeurs uniques de y_test : [0 1 2 3]


In [ ]:

from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier()

In [ ]:
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200]
}


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, roc_auc_score, accuracy_score

In [ ]:
scorer = make_scorer(recall_score, average = "macro", labels = [1])

In [ ]:
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring= scorer,error_score = "raise")

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [3, 6, 9],
                         'n_estimators': [50, 100, 200]},
             scoring=make_scorer(recall_score, average=macro, labels=[1]))

In [ ]:
print("Best parameters:", grid_search.best_params_)

Best parameters: {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 100}


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print("Classification Report sur l'ensemble de test avec XGboost(): \n", classification_report(y_test, y_pred))

Classification Report sur l'ensemble de test avec XGboost(): 
               precision    recall  f1-score   support

           0       0.63      0.74      0.68      6163
           1       0.52      0.65      0.58      6163
           2       0.41      0.29      0.34      6164
           3       0.48      0.44      0.46      6163

    accuracy                           0.53     24653
   macro avg       0.51      0.53      0.51     24653
weighted avg       0.51      0.53      0.51     24653

